# Monte Carlo simulation of X-ray imaging

The two main interaction processes between photon and matter is the photoelectric effect and Compton scattering. The photoelectric effect is a process where an electron with energy E is absorbed by an atom. The process where a photon interacts with an outer electron of an atom, leading to a change in the photons energy and direction is known as Compton scattering. When taking these two effects into account, we assume that there is a probability p of a photon being absorbed or scattered when moving a small distance x to $x+\Delta x$. (And a probability of 1-p of it moving through unhindered.)
If we assume that the intensity of a photon beam is equal to $I(x)$, the above assumption would mean that the expected intensity of the beam at $x + \Delta x$ is equal to 

$I(x+\Delta x) = I(x)(1-p) + 0*p \,\,\,\,(1)$.

The first part of this notebook will look at x-ray imaging in one direction. The space it moves through is discretized into N different sections, where each distance is given by $\Delta x$. In this project the assumption that $p = \mu \Delta x$ will be made, where $\mu$ is the so-called attenuation coefficient. The attenuation coefficient describes the extent at which the radiant flux of a beam reduces as it moves through a material.

Rearranging the terms in (1), plugging in the definition of the probability p, and considering infitesimal steps ($\Delta x \rightarrow 0$), we get the differential equation

$\frac{dI(x)}{dx} = -\mu I(x) \,\,\,\,(2)$.

This equation has the analytical solution 

$I(x) = I_0 e^{-\mu x} \,\,\,\, (3)$,

where $I_0$ is the initial intensity of the beam. This analytical solution will be used to measure how good the numerical approximation is.

For the later parts of the project, some data of attenuation coefficients will be used. These are structured in datafiles giving the attenuation coefficient of the different materials for different energies.

In [1]:
%matplotlib notebook
# Importing necessary libraries
import numpy as np # Numpy for handling arrays
import matplotlib.pyplot as plt # Matplotlib to do the plotting
from mpl_toolkits.mplot3d import Axes3D # For 3D plots
from IPython.display import HTML, display # For tables
import tabulate # Tables

# Fixing some parameters for all of the figures
plt.rcParams.update({'axes.grid': True, 'grid.linestyle' : "--", 'figure.figsize' : (9.5,6)})

In [2]:
%%html 
<style>
.output_wrapper button.btn.btn-default, .output_wrapper .ui-dialog-titlebar {
    display: none;
} </style>

Above block is just to remove the interactive buttons that comes with the notebook backend to get cleaner figures.

### Part 1

In this part a one-dimensional beam of photons will be sent through a material with dampening coefficient, $\mu$.
The result from the Monte-Carlo method and the analytical solution will be compared.
In the end the stability of the Monte-Carlo method will be looked into by comparing different number of photons and different steplengths $\Delta x$. This part is mainly to check that the numerical solver corresponds with the analytical solution.

In [3]:
def simulate_photons(n_photons, n_steps, width, my):
    """
    Approximates the intensity of a photon beam moving through a material evaluated at different steps.
    The approximation is based on a Monte Carlo method using the fact that for each distance x to x + Delta x
    there is a probability p that the photon either will scatter or be absorved. This probability is equal to
    the length moved and the attenuation coefficient. A random number is sampled at each step and compared to this 
    probability to determine if the photon will keep on moving or not.

        Parameters:
            n_photons (int):        number of photons
            n_steps (int):          number of discrete steps taken through the material
            width (float):          the width of the material (cm)
            my (1D float array):    the attenuation coefficient of the material at each step
        
        Output:
            I (1D float array):     the intensity at each step relative to initial intensity
    """
    dx = width / n_steps
    lengths = np.zeros(n_photons) # Each element i is the lenght (cm) photon i reached
    p = my * dx # Pobabilities of being scattered/absorbed at the different steps           
    r = np.random.rand(n_photons, n_steps) # Random numbers for comparing with p
    
    for i in range(n_photons):    
        # Simulate each photon seperately      
        length = width # If not scattered/absorbed this won't be overwritten
        for j in range(n_steps):
            # Going through each step in the material
            if r[i, j] < p[j]:
                # If true the photon was absorbed during step j
                length = dx * j # The length the photon reached
                break
        lengths[i] = length # Storing the length photon i reached
    
    I = np.zeros(n_steps + 1) # Contains the relative intensity at each gridpoint (including start/end)

    I[0] = 1 # First intensity equal to 1
    for i in range(n_steps):
        # Calculating the intensity after each step
        nFotoner_i = np.sum(lengths == dx * i) # The number of photons absorbed in this step
        I[i+1] = I[i] - nFotoner_i / n_photons # Removing the percentage of photons absorbed from intensity
    
    return I

In [4]:
# Defining some parameters for the first part
# Adding _1 to the parameters to keep them separated from the other parts
'''
- width: the width of the material (cm)
- n_steps: the number of steps
- x: the points along the x-axis
- n_photons: number of photons
- my: the attenuation coefficient of the material, cm^-1
'''

width_1 = 10
n_steps_1 = 100
x_1 = np.linspace(0, width_1, n_steps_1 + 1)
n_photons_1 = 1000
my1 = 0.1 * np.ones(n_steps_1)

In [5]:
I1 = simulate_photons(n_photons_1, n_steps_1, width_1, my1) 
print(I1)

[1.    0.991 0.979 0.97  0.962 0.952 0.943 0.93  0.924 0.919 0.91  0.9
 0.888 0.879 0.873 0.869 0.859 0.848 0.84  0.829 0.818 0.813 0.807 0.799
 0.789 0.778 0.773 0.767 0.757 0.751 0.746 0.733 0.726 0.72  0.712 0.706
 0.7   0.691 0.689 0.68  0.675 0.669 0.662 0.659 0.65  0.64  0.632 0.629
 0.622 0.618 0.611 0.607 0.604 0.6   0.589 0.584 0.581 0.574 0.565 0.561
 0.557 0.554 0.546 0.536 0.529 0.526 0.521 0.516 0.511 0.504 0.499 0.493
 0.491 0.485 0.48  0.474 0.47  0.467 0.463 0.459 0.454 0.451 0.45  0.443
 0.441 0.432 0.428 0.426 0.425 0.418 0.412 0.405 0.4   0.393 0.391 0.387
 0.383 0.378 0.373 0.37  0.365]


### Part 2

In this part two different photon beams will be simulated. One will go through only tissue, and the other will first go through tissue, then through bone and in the end through tissue again. Their contrast will be compared at different energy levels.

### Part 3
In this part photon beams will be sent through different 3d objects at different energies. 2d images will be created by sending the photon beams through the three different axise.